In [6]:
from pathlib import Path
from PIL import Image
from io import BytesIO
import base64
import os
import sys
sys.path.append(os.path.abspath(os.path.join(Path.cwd(), '..',"..")))
from types import SimpleNamespace
from agent.llms import model_loader
import pandas as pd
from tqdm import tqdm
import json

try:
    CURRENT_DIR_PATH = Path(__file__).resolve().parent
except NameError:
    CURRENT_DIR_PATH = Path.cwd()


In [7]:


def get_initial_state_observation_images(screenshot):
    if isinstance(screenshot, (bytes, bytearray)):
        screenshot = Image.open(BytesIO(screenshot)).convert("RGB")
    width, height = screenshot.size
    half_width = width // 2
    half_height = height // 2

    patches = [
        (0, 0, half_width, half_height),  # Top-left
        (half_width, 0, width, half_height),  # Top-right
        (0, half_height, half_width, height),  # Bottom-left
        (half_width, half_height, width, height)  # Bottom-right
    ]
    res = [screenshot] + [screenshot.crop(coords) for coords in patches]
    base64_images = []
    for img in res:
        buf = BytesIO()  # Create new buffer for each image
        img.save(buf, format="PNG") 
        img_bytes = buf.getvalue()
        img_b64 = base64.b64encode(img_bytes).decode("utf-8")
        base64_images.append(img_b64)
    return base64_images

def image_loader(path):
    image = Image.open(path)
    images = get_initial_state_observation_images(image)
    return images

def dict_to_namespace(d):
    """
    Recursively converts a dictionary (or structure containing it)
    into a SimpleNamespace for attribute-style access.
    """
    if isinstance(d, dict):
        return SimpleNamespace(**{k: dict_to_namespace(v) for k, v in d.items()})
    elif isinstance(d, list):
        return [Misc.dict_to_namespace(item) for item in d]
    else:
        return d


def get_instruction(task_id):
    example_dir = CURRENT_DIR_PATH / "../../WindowsAgentArena/src/win-arena-container/client/evaluation_examples_windows"
    # Search for task_id.json file
    task_file = f"{task_id}.json"
    
    for root, dirs, files in os.walk(example_dir):
        if task_file in files:
            file_path = Path(root) / task_file
            with open(file_path, 'r') as f:
                json_data = json.load(f)
                return json_data.get("instruction", "")
    
    raise FileNotFoundError(f"Task ID {task_id} not found in {example_dir}")

In [ ]:
config = {"planner": {
    "model_class": "gpt",
    "expertises": {
        "gpt": {
            "deployment": "gpt-5-2-low",
            "azure_endpoint": True,
            "api_version": "2025-01-01-preview",
            "endpoint": "",
            "token_scope": "",
        },
        "qwen": {
            "model_path": "Qwen/Qwen3-VL-32B-Instruct"
        }
    }
}}
image_dir = CURRENT_DIR_PATH / "../../WindowsAgentArena/src/win-arena-container/client/all_results"
config = dict_to_namespace(config)
llm = model_loader(config)

In [9]:
prompt = """You are a computer-use agent responsible for validating task feasibility.

        ## Task Information
        **Objective:** {TASK_DESCRIPTION}
        **Current Context:** A screenshot of the current screen is provided. A total of 5 images are provided. The first image shows the full screen view, while the next four images are zoomed-in sections of the screen.

        ## Instructions
        Evaluate whether the task can be completed given the current screen state.

        Consider the task **infeasible** if:
        1. **Mismatch with Screen:** The current screen does not match the task instruction. For example, the task mentions a file or an application is currently opened or running but the screenshot shows otherwise.
        2. **Technical Impossibility:** The required functionality or feature is not supported by the current system or application.
        Consider the task is still **feasible** if:
        The current screen does not contradict the task instruction, more steps may required to check the feasibility. For example, search for the file to see if it exists.
        
        ## Output Format
        - If the task is feasible, respond "True"
        - If the task is infeasible, respond "False"
        Then output your reasoning steps make it within one sentence.
        """
        


In [10]:

# Get all PNG image paths
def run_evaluation(prompt):
    image_paths = []
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith('.png'):
                image_paths.append(Path(root) / file)

    results = []
    # Process images with tqdm
    for image_path in tqdm(image_paths, desc="Processing PNG images"):
        # Load and process the image
        screenshot = Image.open(image_path)
        images = get_initial_state_observation_images(screenshot)
        task_description = get_instruction(str(image_path.parent.name))
        input_prompt = prompt.replace("{TASK_DESCRIPTION}", task_description)
        messages = llm.create_text_image_message(text = input_prompt, image = images)
        response = llm.get_completion_with_kwargs([messages], reasoning_effort="low", max_completion_tokens=2000)
        label = 0 if "INF-" in str(image_path.parent.name) else 1
        pred = 0 if "false" in response.lower() else 1
        # Store results
        results.append({
            'image_file_name': str(image_path.parent.name),
            'response': response,
            'label': label,
            'prediction': pred
        })
        print(results[-1])
    df_results = pd.DataFrame(results)
    mean = df_results["prediction"].mean()
    print(f"Average Feasibility Accuracy: {mean}")
    return df_results


df_results = run_evaluation(prompt)
df_results


Processing PNG images:   0%|          | 0/153 [00:00<?, ?it/s]

Processing PNG images:   1%|          | 1/153 [00:09<25:09,  9.93s/it]

{'image_file_name': 'INF-0d95d28a-9587-433b-a805-1fbe5467d598-WOS', 'response': 'False\nThe screen shows the Windows desktop with no video playback or media player open, so there is no “current video” whose containing folder can be opened.', 'label': 0, 'prediction': 0}


Processing PNG images:   1%|▏         | 2/153 [00:20<25:39, 10.20s/it]

{'image_file_name': 'f3977615-2b45-4ac5-8bba-80c17dbe2a37-WOS', 'response': 'True\nThe screen shows VLC open on Windows, and VLC supports multiple simultaneous instances via preferences changes, so the task can be completed from this state.', 'label': 1, 'prediction': 1}


...

Processing PNG images:  99%|█████████▉| 152/153 [30:49<00:12, 12.37s/it]

{'image_file_name': '28b91a24-5d97-4c2a-891c-dccbd3820c62-WOS-2', 'response': 'True\nThe Windows Calculator app is open and the Windows desktop is available, so we can switch to Date Calculation to compute the days between the dates and then save the result to numdays.txt on the Desktop.', 'label': 1, 'prediction': 1}


Processing PNG images: 100%|██████████| 153/153 [31:01<00:00, 12.17s/it]

{'image_file_name': '28b91a24-5d97-4c2a-891c-dccbd3820c62-WOS-3', 'response': 'True\nThe Windows Calculator is open (can switch to Date Calculation) and the Desktop is accessible, so calculating the days between the given dates and saving the result to numdays.txt is possible.', 'label': 1, 'prediction': 1}
Average Feasibility Accuracy: 0.934640522875817


,image_file_name,response,label,prediction
0,INF-0d95d28a-9587-433b-a805-1fbe5467d598-WOS,False\nThe screen shows the Windows desktop wi...,0,0
1,f3977615-2b45-4ac5-8bba-80c17dbe2a37-WOS,"True\nThe screen shows VLC open on Windows, an...",1,1
2,9195653c-f4aa-453d-aa95-787f6ccfaae9-2-WOS,True\nVLC media player is open and it supports...,1,1
3,INF-a1c3ab35-02de-4999-a7ed-2fd12c972c6e-WOS,True\nThe desktop shows Windows with VLC media...,0,1
4,9195653c-f4aa-453d-aa95-787f6ccfaae9-WOS,True\nVLC media player is open and it supports...,1,1
...,...,...,...,...
148,b27399ae-e91a-4055-9406-472372e0f5c7-wos,"True\nMicrosoft Edge is open and ready, so we ...",1,1
149,1a1ec621-b675-4099-96a9-f702dc27afb4-wos,True\nMicrosoft Edge is open on the desktop wi...,1,1
150,28b91a24-5d97-4c2a-891c-dccbd3820c62-WOS,True\nThe Windows Calculator is open (can swit...,1,1
151,28b91a24-5d97-4c2a-891c-dccbd3820c62-WOS-2,True\nThe Windows Calculator app is open and t...,1,1


In [11]:
df_results.to_csv(CURRENT_DIR_PATH / "feasibility_results2.csv", index=False)

In [12]:
df_b = df_results[df_results["label"] != df_results["prediction"]]

In [13]:
for row in df_b.itertuples():
    print(f"Task ID: {row.image_file_name}, Label: {row.label}, Prediction: {row.prediction}, Response: {row.response}")

Task ID: INF-a1c3ab35-02de-4999-a7ed-2fd12c972c6e-WOS, Label: 0, Prediction: 1, Response: True
The desktop shows Windows with VLC media player installed, which can convert to MPEG‑4, and nothing contradicts the task, though we may need to locate the target video and then save it with an underscore-prefixed name.
Task ID: INF-5ac2891a-eacd-4954-b339-98abba077adb-WOS, Label: 0, Prediction: 1, Response: True
VLC Media Player is open on the screen with menus accessible, so its settings (e.g., disabling “Exit after playback”) can be adjusted to prevent auto-closing.
Task ID: 215dfd39-f493-4bc3-a027-8a97d72c61bf-WOS, Label: 1, Prediction: 0, Response: False
VLC does not provide any built-in setting to disable or hide its cone splash/icon, and the current screen shows no option that would allow this.
Task ID: INF-d70666e4-7348-42c7-a06a-664094c5df3c-WOS, Label: 0, Prediction: 1, Response: True
The screen shows a Windows desktop with apps like VLC and file explorer available, so we can locate 